# Proyek Klasifikasi Gambar: "Honkai: Star Rail" Map
- **Nama:** Muhammad Faisal Aktsa
- **Email:** fslfaisal17@gmail.com
- **ID Dicoding:** fslmc17

## Import Semua Packages/Library yang Digunakan

## Data Preparation

### Data Loading

### Data Preprocessing

#### Split Dataset

## Modelling

## Evaluasi dan Visualisasi

## Konversi Model

## Inference (Optional)